In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%set_env CUDA_LAUNCH_BLOCKING=1

env: CUDA_LAUNCH_BLOCKING=1


In [3]:
import sys
import collections
import cv2
import os
import math
import random
import pickle
import copy
import numpy as np
import pandas as pd
sys.path.append("C:\\Users\\user\\Documents\\Workspace\\MeronymNet-PyTorch\\src")

In [4]:
import torch
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.utils import dense_to_sparse
from torch_geometric.data import DataLoader
import torch.utils.data as data_utils
from torch.utils.tensorboard import SummaryWriter

In [6]:
from losses import BoxVAE_losses as loss
from evaluation import metrics
from utils import plot_utils
from utils import data_utils as data_loading
from components.DenseAutoencoder import DenseAutoencoder
from components.DenseAutoencoder import Decoder
from components.TwoStageAutoEncoder import TwoStageAutoEncoder

In [5]:
import matplotlib.pyplot as plt

In [7]:
# from mask_generation import masked_sketch

In [20]:
batch_size = 128
latent_dims = 64
num_nodes = 16
bbx_size = 4
num_classes = 7
label_shape = 1
nb_epochs = 1000
klw = loss.frange_cycle_linear(nb_epochs)
learning_rate = 0.00005
hidden1 = 32
hidden2 = 16
hidden3 = 128
dense_hidden1=8
dense_hidden2=4
adaptive_margin = True
fine_tune_box = False
output_log = False
area_encoding = False
run_prefix = "two_stage_small_obj_conditioning"
variational=False
coupling = True
obj_bbx_conditioning = True

In [9]:
batch_train_loader, batch_val_loader = data_loading.load_data(obj_data_postfix = '_obj_boundary',
                                                              part_data_post_fix = '_scaled',
                                                              file_postfix = '_combined',
                                                              seed=345,
                                                              batch_size=batch_size)

C:\Users\user\miniconda3\lib\site-packages\torch_geometric\deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [10]:
import gc
gc.collect()

21

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [21]:
reconstruction_loss_arr = []
kl_loss_obj_arr = []
kl_loss_part_arr = []
bbox_loss_arr = []
adj_loss_arr = []
node_loss_arr = []

reconstruction_loss_val_arr = []
kl_loss_val_arr = []
bbox_loss_val_arr = []
adj_loss_val_arr = []
node_loss_val_arr = []

vae = TwoStageAutoEncoder(latent_dims,
                          num_nodes,
                          bbx_size,
                          num_classes,
                          label_shape,
                          hidden1,
                          hidden2,
                          hidden3,
                          dense_hidden1,
                          dense_hidden2,
                          adaptive_margin,
                          output_log,
                          area_encoding,
                          coupling,
                          obj_bbx_conditioning
                        )
vae.to(device)
optimizer = optim.Adam(vae.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[200,400], gamma=0.75)
model_path = ('D:/meronym_data/model/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')
summary_path = ('D:/meronym_data/runs/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')
if not os.path.exists(model_path):
    os.makedirs(model_path)
writer = SummaryWriter(summary_path)
icoef = 0

for epoch in range(nb_epochs):
    
    batch_loss = torch.tensor([0.0])
    batch_kl_loss_part = torch.tensor([0.0])
    batch_kl_loss_obj = torch.tensor([0.0])
    batch_bbox_loss = torch.tensor([0.0])
    batch_obj_bbox_loss = torch.tensor([0.0])
    batch_node_loss = torch.tensor([0.0])
    IOU_weight_delta = torch.tensor([(1+epoch)/nb_epochs])
    images = []
    
    vae.train()
    i=0
    for train_data in batch_train_loader:
        
        node_data_true = train_data.x
        label_true = node_data_true[:,:1]
        y_true = train_data.y
        class_true = y_true[:, :num_classes]
        X_obj_true = y_true[:, num_classes:]
        X_obj_true_transformed = X_obj_true[:,2:]-X_obj_true[:,:2]
        adj_true = train_data.edge_index
        batch = train_data.batch
        
        class_true  = torch.flatten(class_true)
        

        for param in vae.parameters():
            param.grad=None
        
        output = vae(adj_true, node_data_true, X_obj_true_transformed, label_true , class_true, False, coupling)
        
        node_data_pred = output[0]
        X_obj_pred = output[1]
        X_obj_pred = torch.cat(((torch.tensor([1.0]).cuda()-X_obj_pred)*torch.tensor([0.5]).cuda(),
                                (torch.tensor([1.0]).cuda()+X_obj_pred)*torch.tensor([0.5]).cuda()),
                              -1)
        label_pred = output[2]
        z_mean_part = output[3]
        z_logvar_part = output[4]
        margin = output[5]
        z_mean_obj = output[6]
        z_logvar_obj = output[7]
        
        obj_bbox_loss = loss.obj_bbox_loss(pred_box=X_obj_pred, true_box=X_obj_true, weight=IOU_weight_delta, has_mse=False)
        kl_loss_obj = loss.kl_loss(z_mean_obj, z_logvar_obj)
        kl_loss_part = loss.kl_loss(z_mean_part, z_logvar_part)
        bbox_loss = loss.weighted_bbox_loss(pred_box=node_data_pred, true_box=node_data_true[:,1:], weight=IOU_weight_delta, margin=margin)
        node_loss = loss.node_loss(label_pred,label_true)
        
        kl_weight = klw[icoef]
        
        if kl_weight>0:
            reconstruction_loss = kl_loss*kl_weight + (bbox_loss + node_loss)*num_nodes
        else:
            reconstruction_loss = (bbox_loss + node_loss)*num_nodes
            
        reconstruction_loss.backward()
        
        optimizer.step()
        
        i+=1
      
        batch_loss += reconstruction_loss
        batch_kl_loss_part += kl_loss_part
        batch_kl_loss_obj += kl_loss_obj
        batch_bbox_loss += bbox_loss
        batch_obj_bbox_loss += obj_bbox_loss
        batch_node_loss += node_loss
    
        if i%200==0:
            print(i)
            global_step = epoch*len(batch_train_loader)+i
            
            writer.add_scalar("Loss/train/reconstruction_loss", batch_loss.item()/(i+1), global_step)
            writer.add_scalar("Loss/train/kl_loss_part", batch_kl_loss_part.item()/(i+1), global_step)
            writer.add_scalar("Loss/train/kl_loss_obj", batch_kl_loss_obj.item()/(i+1), global_step)
            writer.add_scalar("Loss/train/bbox_loss", batch_bbox_loss.item()/(i+1), global_step)
            writer.add_scalar("Loss/train/obj_bbox_loss", batch_obj_bbox_loss.item()/(i+1), global_step)
            writer.add_scalar("Loss/train/node_loss", batch_node_loss.item()/(i+1), global_step)
            
            
#     scheduler.step()
    global_step = epoch*len(batch_train_loader)+i
    image_shape = [num_nodes, bbx_size]

    image = plot_utils.plot_bbx(np.reshape((node_data_true[:num_nodes,1:5]*label_true[:num_nodes]).detach().to("cpu").numpy(),
                                image_shape)).astype(float)/255
    writer.add_image('train/images/input', image, global_step, dataformats='HWC')
    image = plot_utils.plot_bbx((node_data_pred[0]*label_true[:num_nodes]).detach().to("cpu").numpy()).astype(float)/255
    writer.add_image('train/images/generated', image, global_step, dataformats='HWC')
    
    reconstruction_loss_arr.append(batch_loss.detach().item()/(i+1))
    kl_loss_obj_arr.append(batch_kl_loss_obj.detach().item()/(i+1))
    kl_loss_part_arr.append(batch_kl_loss_part.detach().item()/(i+1))
    bbox_loss_arr.append(batch_bbox_loss.detach().item()/(i+1))
    node_loss_arr.append(batch_node_loss.detach().item()/(i+1))
    
    print('[%d, %5d] loss: %.3f' %
              (epoch + 1, i + 1, batch_loss/(i+1) ))
    
    
    batch_loss = torch.tensor([0.0])
    batch_kl_loss_part = torch.tensor([0.0])
    batch_kl_loss_obj = torch.tensor([0.0])
    batch_bbox_loss = torch.tensor([0.0])
    batch_obj_bbox_loss = torch.tensor([0.0])
    batch_node_loss = torch.tensor([0.0])
    images = []
    vae.eval()
    for i, val_data in enumerate(batch_val_loader, 0):
        
        node_data_true = val_data.x
        label_true = node_data_true[:,:1]
        y_true = val_data.y
        class_true = torch.flatten(y_true[:, :num_classes])
        X_obj_true = y_true[:, num_classes:]
        X_obj_true_transformed = X_obj_true[:,2:]-X_obj_true[:,:2]
        adj_true = val_data.edge_index
        batch = val_data.batch
        
        class_true  = torch.flatten(class_true)
        
        output = vae(adj_true, node_data_true, X_obj_true_transformed, label_true , class_true, False, coupling)
        
        node_data_pred = output[0]
        X_obj_pred = output[1]
        X_obj_pred = torch.cat(((torch.tensor([1.0]).cuda()-X_obj_pred)*torch.tensor([0.5]).cuda(),
                                (torch.tensor([1.0]).cuda()+X_obj_pred)*torch.tensor([0.5]).cuda()),
                              -1)
        label_pred = output[2]
        z_mean_part = output[3]
        z_logvar_part = output[4]
        margin = output[5]
        z_mean_obj = output[6]
        z_logvar_obj = output[7]
        
        obj_bbox_loss = loss.obj_bbox_loss(pred_box=X_obj_pred, true_box=X_obj_true, weight=IOU_weight_delta, has_mse=False)
        kl_loss_obj = loss.kl_loss(z_mean_obj, z_logvar_obj)
        kl_loss_part = loss.kl_loss(z_mean_part, z_logvar_part)
        bbox_loss = loss.weighted_bbox_loss(pred_box=node_data_pred, true_box=node_data_true[:,1:], weight=IOU_weight_delta, margin=margin)
        node_loss = loss.node_loss(label_pred,label_true)
        
        if kl_weight>0:
            reconstruction_loss = kl_loss*kl_weight + (bbox_loss + node_loss)*num_nodes
        else:
            reconstruction_loss = (bbox_loss + node_loss)*num_nodes
            
        batch_loss += reconstruction_loss
        batch_kl_loss_part += kl_loss_part
        batch_kl_loss_obj += kl_loss_obj
        batch_bbox_loss += bbox_loss
        batch_obj_bbox_loss += obj_bbox_loss
        batch_node_loss += node_loss
    
    image = plot_utils.plot_bbx(np.reshape((node_data_true[:num_nodes,1:5]*label_true[:num_nodes]).detach().to("cpu").numpy(),
                                image_shape)).astype(float)/255
    writer.add_image('val/images/input', image, global_step, dataformats='HWC')
    image = plot_utils.plot_bbx((node_data_pred[0]*label_true[:num_nodes]).detach().to("cpu").numpy()).astype(float)/255
    writer.add_image('val/images/generated', image, global_step, dataformats='HWC')

    reconstruction_loss_val_arr.append(batch_loss.detach().item()/(i+1))
    bbox_loss_val_arr.append(batch_bbox_loss.detach().item()/(i+1))
    node_loss_val_arr.append(batch_node_loss.detach().item()/(i+1))
    
    writer.add_scalar("Loss/val/reconstruction_loss", batch_loss.detach()/(i+1), global_step)
    writer.add_scalar("Loss/train/kl_loss_part", batch_kl_loss_part.item()/(i+1), global_step)
    writer.add_scalar("Loss/train/kl_loss_obj", batch_kl_loss_obj.item()/(i+1), global_step)
    writer.add_scalar("Loss/val/bbox_loss", batch_bbox_loss.item()/(i+1), global_step)
    writer.add_scalar("Loss/val/obj_bbox_loss", batch_obj_bbox_loss.item()/(i+1), global_step)
    writer.add_scalar("Loss/val/node_loss", batch_node_loss.item()/(i+1), global_step)
       
    if epoch%50 == 0:
        torch.save(vae.state_dict(), model_path + '/model_weights.pth')
        
#     if kl_loss_arr[-1]>0.5 and abs(bbox_loss_arr[-1] - bbox_loss_val_arr[-1]) < 0.02 and bbox_loss_arr[-1]<1.3 and epoch>60:
#         icoef = icoef + 1  

torch.save(vae.state_dict(),model_path + '/model_weights.pth')

for i in range(min(100,int(len(node_data_true)/num_nodes))):    
    image = plot_utils.plot_bbx(np.reshape((node_data_true[num_nodes*i:num_nodes*(i+1),1:5]).detach().to("cpu").numpy(),
                                    image_shape)).astype(float)/255
    writer.add_image('result/images/'+str(i)+'-input', image, global_step, dataformats='HWC')
    image = plot_utils.plot_bbx((node_data_pred[i]*label_true[num_nodes*i:num_nodes*(i+1)]).detach().to("cpu").numpy()).astype(float)/255
    writer.add_image('result/images/'+str(i)+'-generated', image, global_step, dataformats='HWC')
    
writer.flush()
writer.close()
print('Finished Training')

200
[1,   399] loss: 123.798
200
[2,   399] loss: 93.176
200
[3,   399] loss: 90.826
200
[4,   399] loss: 90.306
200
[5,   399] loss: 89.907
200
[6,   399] loss: 89.530
200
[7,   399] loss: 89.135
200
[8,   399] loss: 88.716
200
[9,   399] loss: 88.126
200
[10,   399] loss: 86.605
200
[11,   399] loss: 83.180
200
[12,   399] loss: 79.374
200
[13,   399] loss: 76.940
200
[14,   399] loss: 75.501
200
[15,   399] loss: 74.228
200
[16,   399] loss: 72.798
200
[17,   399] loss: 71.196
200
[18,   399] loss: 69.348
200
[19,   399] loss: 66.514
200
[20,   399] loss: 61.641
200
[21,   399] loss: 57.688
200
[22,   399] loss: 55.115
200
[23,   399] loss: 53.378
200
[24,   399] loss: 51.697
200
[25,   399] loss: 49.435
200
[26,   399] loss: 46.722
200
[27,   399] loss: 44.421
200
[28,   399] loss: 42.808
200
[29,   399] loss: 41.672
200
[30,   399] loss: 40.802
200
[31,   399] loss: 40.099
200
[32,   399] loss: 39.498
200
[33,   399] loss: 38.960
200
[34,   399] loss: 38.458
200
[35,   399] loss: 

200
[278,   399] loss: 9.747
200
[279,   399] loss: 9.711
200
[280,   399] loss: 9.675
200
[281,   399] loss: 9.638
200
[282,   399] loss: 9.603
200
[283,   399] loss: 9.567
200
[284,   399] loss: 9.531
200
[285,   399] loss: 9.496
200
[286,   399] loss: 9.461
200
[287,   399] loss: 9.426
200
[288,   399] loss: 9.391
200
[289,   399] loss: 9.356
200
[290,   399] loss: 9.322
200
[291,   399] loss: 9.288
200
[292,   399] loss: 9.254
200
[293,   399] loss: 9.220
200
[294,   399] loss: 9.186
200
[295,   399] loss: 9.153
200
[296,   399] loss: 9.120
200
[297,   399] loss: 9.088
200
[298,   399] loss: 9.055
200
[299,   399] loss: 9.023
200
[300,   399] loss: 8.990
200
[301,   399] loss: 8.959
200
[302,   399] loss: 8.927
200
[303,   399] loss: 8.896
200
[304,   399] loss: 8.865
200
[305,   399] loss: 8.834
200
[306,   399] loss: 8.803
200
[307,   399] loss: 8.773
200
[308,   399] loss: 8.743
200
[309,   399] loss: 8.713
200
[310,   399] loss: 8.684
200
[311,   399] loss: 8.655
200
[312,   39

200
[561,   399] loss: 4.574
200
[562,   399] loss: 4.565
200
[563,   399] loss: 4.555
200
[564,   399] loss: 4.545
200
[565,   399] loss: 4.535
200
[566,   399] loss: 4.525
200
[567,   399] loss: 4.515
200
[568,   399] loss: 4.505
200
[569,   399] loss: 4.496
200
[570,   399] loss: 4.486
200
[571,   399] loss: 4.476
200
[572,   399] loss: 4.466
200
[573,   399] loss: 4.457
200
[574,   399] loss: 4.447
200
[575,   399] loss: 4.438
200
[576,   399] loss: 4.429
200
[577,   399] loss: 4.419
200
[578,   399] loss: 4.410
200
[579,   399] loss: 4.401
200
[580,   399] loss: 4.392
200
[581,   399] loss: 4.382
200
[582,   399] loss: 4.373
200
[583,   399] loss: 4.363
200
[584,   399] loss: 4.354
200
[585,   399] loss: 4.345
200
[586,   399] loss: 4.336
200
[587,   399] loss: 4.327
200
[588,   399] loss: 4.318
200
[589,   399] loss: 4.309
200
[590,   399] loss: 4.300
200
[591,   399] loss: 4.291
200
[592,   399] loss: 4.281
200
[593,   399] loss: 4.273
200
[594,   399] loss: 4.264
200
[595,   39

200
[844,   399] loss: 2.743
200
[845,   399] loss: 2.738
200
[846,   399] loss: 2.734
200
[847,   399] loss: 2.730
200
[848,   399] loss: 2.727
200
[849,   399] loss: 2.722
200
[850,   399] loss: 2.718
200
[851,   399] loss: 2.715
200
[852,   399] loss: 2.711
200
[853,   399] loss: 2.707
200
[854,   399] loss: 2.701
200
[855,   399] loss: 2.697
200
[856,   399] loss: 2.693
200
[857,   399] loss: 2.689
200
[858,   399] loss: 2.684
200
[859,   399] loss: 2.680
200
[860,   399] loss: 2.676
200
[861,   399] loss: 2.672
200
[862,   399] loss: 2.668
200
[863,   399] loss: 2.664
200
[864,   399] loss: 2.660
200
[865,   399] loss: 2.656
200
[866,   399] loss: 2.652
200
[867,   399] loss: 2.648
200
[868,   399] loss: 2.644
200
[869,   399] loss: 2.640
200
[870,   399] loss: 2.635
200
[871,   399] loss: 2.631
200
[872,   399] loss: 2.627
200
[873,   399] loss: 2.623
200
[874,   399] loss: 2.620
200
[875,   399] loss: 2.616
200
[876,   399] loss: 2.612
200
[877,   399] loss: 2.607
200
[878,   39

In [13]:
vae

TwoStageAutoEncoder(
  (gcn_encoder): GCNEncoder(
    (gconv1): GCNConv(5, 32)
    (gconv2): GCNConv(32, 16)
    (dense_boxes): Linear(in_features=4, out_features=16, bias=True)
    (dense_labels): Linear(in_features=1, out_features=16, bias=True)
    (act): ReLU()
    (dense1): Linear(in_features=256, out_features=128, bias=True)
    (dense2): Linear(in_features=263, out_features=128, bias=True)
    (dense3): Linear(in_features=128, out_features=128, bias=True)
    (latent): Linear(in_features=128, out_features=64, bias=True)
  )
  (margin_layer): Linear(in_features=8, out_features=2, bias=True)
  (margin_activation): Sigmoid()
  (gcn_decoder): Decoder(
    (dense1): Linear(in_features=89, out_features=128, bias=True)
    (dense2): Linear(in_features=128, out_features=128, bias=True)
    (dense_bbx): Linear(in_features=128, out_features=64, bias=True)
    (dense_lbl): Linear(in_features=128, out_features=16, bias=True)
    (dense_edge): Linear(in_features=128, out_features=256, bias=T

In [22]:
#testing loop
model_path = ('D:/meronym_data/model/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')
summary_path = ('D:/meronym_data/runs/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')


write_tensorboard = False
if write_tensorboard:
    writer = SummaryWriter(summary_path)

vae = TwoStageAutoEncoder(latent_dims,
                          num_nodes,
                          bbx_size,
                          num_classes,
                          label_shape,
                          hidden1,
                          hidden2,
                          hidden3,
                          dense_hidden1,
                          dense_hidden2,
                          adaptive_margin,
                          output_log,
                          area_encoding,
                          coupling,
                          obj_bbx_conditioning,
                        )

vae.load_state_dict(torch.load(model_path+ '/model_weights.pth'))

# decoder = vae.decoder
image_shape = [num_nodes, bbx_size]
global_step = 250000
class_dict = {0:'cow', 1:'sheep', 2:'bird', 3:'person', 4:'cat', 5:'dog', 6:'horse'}
res_dfs = []
for i, val_data in enumerate(batch_val_loader, 0):
    
    val_data.cuda()
    node_data_true = val_data.x
    label_true = node_data_true[:,:1]
    y_true = val_data.y
    class_true = y_true[:, :num_classes]
    X_obj_true = y_true[:, num_classes:]
    X_obj_true_transformed = X_obj_true[:,2:]-X_obj_true[:,:2]
    adj_true = val_data.edge_index
    class_true  = torch.flatten(class_true)
    
    output = vae(adj_true, node_data_true, X_obj_true_transformed, label_true , class_true, False, True)
    node_data_pred_test = output[0]
    X_obj_pred_test = output[1]
#     node_data_pred_test, X_obj_pred_test, label_pred, z_mean, z_logvar, margin = output
    X_obj_pred_test = torch.cat(((torch.tensor([1.0]).cuda()-X_obj_pred_test)*torch.tensor([0.5]).cuda(),
                                (torch.tensor([1.0]).cuda()+X_obj_pred_test)*torch.tensor([0.5]).cuda()),
                              -1)
    res_dfs.append(metrics.get_metrics(node_data_true, X_obj_true, node_data_pred_test,
                               X_obj_pred_test,
                               label_true,
                               class_true,
                               num_nodes,
                               num_classes))
    
    if write_tensorboard:
        
        for j in range(int(len(node_data_true)/num_nodes)):
            
            image = plot_bbx(node_data_true[j].detach().to("cpu").numpy().astype(float))/255
            pred_image = plot_bbx(node_data_pred_test[j].detach().to("cpu").numpy()).astype(float)/255

            writer.add_image('test_result/images/'+str(j)+'-input/', image, global_step, dataformats='HWC')  
            writer.add_image('test_result/images/'+str(j)+'-generated/', pred_image, global_step, dataformats='HWC')
            

result = pd.concat(res_dfs)
result['obj_class'] = np.where(result['obj_class'].isna(), 0, result['obj_class'])
result['obj_class'] = result['obj_class'].astype('int')
result['obj_class'].replace(class_dict, inplace=True)
result.where(result['part_labels']!=0, np.NaN, inplace=True)
result['part_labels'] = np.where(result['part_labels'].isna(), 0, result['part_labels'])
result['part_labels'] = result['part_labels'].astype('int')
result['id'] = np.repeat(np.array(list(range(int(len(result)/num_nodes)))), 16)

if write_tensorboard:
    writer.flush()
    writer.close()

bird_labels = {'head':1 , 'torso':2, 'neck':3, 'lwing':4, 'rwing':5, 'lleg':6, 'lfoot':7, 'rleg':8, 'rfoot':9, 'tail':10}
cat_labels = {'head':1, 'torso':2, 'neck':3, 'lfleg':4, 'lfpa':5, 'rfleg':6, 'rfpa':7, 'lbleg':8, 'lbpa':9, 'rbleg':10, 'rbpa':11, 'tail':12}
cow_labels = {'head':1,'lhorn':2, 'rhorn':3, 'torso':4, 'neck':5, 'lfuleg':6, 'lflleg':7, 'rfuleg':8, 'rflleg':9, 'lbuleg':10, 'lblleg':11, 'rbuleg':12, 'rblleg':13, 'tail':14}
dog_labels = {'head':1,'torso':2, 'neck':3, 'lfleg':4, 'lfpa':5, 'rfleg':6, 'rfpa':7, 'lbleg':8, 'lbpa':9, 'rbleg':10, 'rbpa':11, 'tail':12, 'muzzle':13}
horse_labels = {'head':1,'lfho':2, 'rfho':3, 'torso':4, 'neck':5, 'lfuleg':6, 'lflleg':7, 'rfuleg':8, 'rflleg':9, 'lbuleg':10, 'lblleg':11, 'rbuleg':12, 'rblleg':13, 'tail':14, 'lbho':15, 'rbho':16}
person_labels = {'head':1, 'torso':2, 'neck': 3, 'llarm': 4, 'luarm': 5, 'lhand': 6, 'rlarm':7, 'ruarm':8, 'rhand': 9, 'llleg': 10, 'luleg':11, 'lfoot':12, 'rlleg':13, 'ruleg':14, 'rfoot':15}
sheep_labels = cow_labels
part_labels_combined_parts = {'bird': bird_labels, 'cat': cat_labels, 'cow': cow_labels, 'dog': dog_labels, 'sheep': sheep_labels, 'horse':horse_labels,'person':person_labels}

for k, v in part_labels_combined_parts.items():
    new_map = {}
    for pk, pv in v.items():
        new_map[pv]=pk
    part_labels_combined_parts[k] = new_map
    
for k, v in part_labels_combined_parts.items():
    result.loc[result['obj_class']==k, ['part_labels']] = result.loc[result['obj_class']==k,['part_labels']].replace(v).copy()

result.to_csv(model_path+'/raw_metrics.csv')
res_obj_level = result.groupby(['obj_class', 'id']).mean().reset_index()
res_obj_level.groupby(['obj_class']).mean().reset_index()[['obj_class', 'IOU', 'MSE']].to_csv(model_path+'/obj_level_metrics.csv')
result.groupby(['obj_class', 'part_labels']).mean().reset_index()[['obj_class', 'part_labels',  'IOU', 'MSE']].to_csv(model_path+'/part_level_metrics.csv')

C:\Users\user\Documents\Workspace\MeronymNet-PyTorch\src\evaluation\metrics.py:21: RuntimeWarning: invalid value encountered in true_divide
  iouk = interArea / unionArea


In [23]:
model_path = ('D:/meronym_data/model/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')
summary_path = ('D:/meronym_data/runs/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')

result = pd.read_csv(model_path+'/raw_metrics.csv')
result.groupby(['obj_class', 'part_labels']).mean().reset_index()[['obj_class', 'part_labels',
                                                                   'IOU', 'MSE']]

,obj_class,part_labels,IOU,MSE
0,bird,head,0.605276,0.000297
1,bird,lfoot,0.288973,0.000531
2,bird,lleg,0.370438,0.000590
3,bird,lwing,0.390736,0.001783
4,bird,neck,0.443385,0.000908
...,...,...,...,...
81,sheep,lhorn,0.398353,0.000691
82,sheep,neck,0.638507,0.000264
83,sheep,rhorn,0.317564,0.000736
84,sheep,tail,0.235653,0.000787


In [24]:
res_obj_level = result.groupby(['obj_class', 'id']).mean().reset_index()
res_obj_level.groupby(['obj_class']).mean().reset_index()[['obj_class', 'IOU', 'MSE']]

,obj_class,IOU,MSE
0,bird,0.539319,0.000580
1,cat,0.598673,0.001588
2,cow,0.539434,0.000497
3,dog,0.487456,0.001050
4,horse,0.431452,0.001060
5,person,0.510287,0.001043
6,sheep,0.676451,0.000334


In [25]:
res_obj_level[['IOU', 'MSE']].mean()

IOU    0.534635
MSE    0.000952
dtype: float64

In [26]:
X_obj_pred[:10,2:]-X_obj_pred[:10,:2]

tensor([[0.6895, 0.3774],
        [0.3707, 0.4534],
        [0.4816, 0.5451],
        [0.7528, 0.5211],
        [0.7168, 0.4764],
        [0.7716, 0.5218],
        [0.3109, 0.2908],
        [0.4015, 0.3818],
        [0.3397, 0.3901],
        [0.6958, 0.4934]])

In [27]:
X_obj_true[:10, 2:]-X_obj_true[:10, :2]

tensor([[0.6895, 0.3774],
        [0.3707, 0.4534],
        [0.4816, 0.5451],
        [0.7528, 0.5211],
        [0.7168, 0.4764],
        [0.7716, 0.5218],
        [0.3109, 0.2908],
        [0.4015, 0.3818],
        [0.3397, 0.3901],
        [0.6958, 0.4934]])

In [ ]:
res_obj_level.to_csv(model_path+'/obj_metrics.csv')

In [ ]:
result.mean().reset_index()

In [ ]:
outfile = 'D:/meronym_data/generate_boxes.npy'
with open(outfile, 'wb') as pickle_file:
    pred_boxes = np.concatenate(pred_boxes)
    pickle.dump(pred_boxes, pickle_file)
outfile = 'D:/meronym_data/generate_boxesobj_class.npy'
with open(outfile, 'wb') as pickle_file:
    pickle.dump(classes,pickle_file)